# Background
Alan Turing and Alonzo Church created models of computation around the same time in the 1930s. Turing's machines are based on a strip of tape while Church's lambda calculus is based on functions. The former is reminiscent of object oriented programming while the latter is a functional programming model. Turing showed that both models are equivalent in showing the power of computers generally.

As such, languages as different as Java and Lisp are *mathematically equivalent* in their capability, given infinite memory and computing power.

For modern programmers, understanding the different modes of programming in a object oriented or functional capacity, can be invaluable in catering to the natural intuition of our minds to solve problems. At times, it's easier to imagine the 'state' of a machine, its memory, and how to transform data from beginning to end. At other times, it's useful to focus on the actions and verbs we enable a computer to execute.

Decorators are a functional programming tool built into Python to more easily modify functions. They are visually distinct in Python and can seem superfluous at first, but like any piece of syntax, can be a powerful tool that simplifies the readability and logic of a program. 

---

# Motivation

In [1]:
def f():
    print('Hello World!')

In [2]:
#f is stand-in for some function.
#Here it print's out 'Hello World!'
f()

Hello World!


In [3]:
#We might be interested in modifying the function in some way.
#For example, executing it twice.
f()
f()

Hello World!
Hello World!


In [4]:
#Alternatively, we could write a second function which is defined
#to execute the first function twice.
def double_f():
    f()
    f()
double_f()

Hello World!
Hello World!


In [5]:
#This isn't great because now we have two functions that need to be kept
#accounted for.
#Alternatively, since Python functions are first-class objects that can be
#passed into other functions, we may modify g to accept another function
#as an argument.
def doubler1(func):
    func()
    func()
    pass
doubler1(f)

Hello World!
Hello World!


In [6]:
#This might suffice if we're comfortable calling f inside of g2
#everytime we wanted this behavior.
#We might also want to modify the behavior of the function without calling
#it right then.
#Making a small adjustment, we can achieve the same behavior of modifying 
#f by having the modifier output the modified function rather.
def doubler2(func):
    def doubler():
        func()
        func()
    return doubler
f2 = doubler2(f)
f2()

Hello World!
Hello World!


In [7]:
#This brings us back to the same problem of having two functions g3 and f 
#cluttering our namespace.
#Still, g3 has the added benefit of allowing us to modify newly written
#functions.
def h():
    print('Good night, world!')
h2 = doubler2(h)
h2()

Good night, world!
Good night, world!


---
# Decorators

In [8]:
#Decorators are functions which modify other functions without
#having to define an intermediary function like g or g2 above.
def star_triangle(char='*', n=3):
    for i in range(n):
        print(char*(i+1))
    pass

#Decorated version
@doubler2
def z_triangle(char='z', n=3):
    for i in reversed(range(n)):
        print(char*(i+1))
    pass

star_triangle()
z_triangle()

*
**
***
zzz
zz
z
zzz
zz
z


In [9]:
#As you might have noticed, z_triangle is being run twice.
#Adding the decorator above the function definition is the
#equivalent of overwriting the original function name with the 
#output of the decorator.
star_triangle = doubler2(star_triangle)
star_triangle()

*
**
***
*
**
***


In [10]:
#Note that while doubler1 gives a doubled *output*, doubler2 gives a
#doubled *function*.
#It's a small and subtle distinction, but it highlights an issue once
#we start passing arguments into the modified function
def star_square(char='*', n=3):
    for i in range(n):
        print(char*(n+1))
    pass
print('Normal execution')
star_square()
print('5x5 square execution with argument')
star_square(n=5)

Normal execution
****
****
****
5x5 square execution with argument
******
******
******
******
******


In [11]:
#Errors out
doubler1(star_square(n=5))

******
******
******
******
******


TypeError: 'NoneType' object is not callable

In [12]:
#Also errors
doubler2(star_square(n=5))()

******
******
******
******
******


TypeError: 'NoneType' object is not callable

In [14]:
#The explanation is that our doubler functions are expecting a function
#as an input, but by providing an argument to our triangle functions
#we are passing its output, a string, into doubler.
#When python tries to execute a string, it errors out because that
#doesn't make sense.
#Using partial is a workaround, but less than ideal when our goal was
#to simplify our syntax rather than making it more obtuse.
from functools import partial
doubler1(partial(star_square, n=5))
#Alternate
doubler2(partial(star_square, char='star ', n=5))()

******
******
******
******
******
******
******
******
******
******
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 
star star star star star star 


---
# Writing Robust Decorators

In [15]:
#Our decorators so far haven't been written to pass through arguments.
z_triangle(char='zee')

TypeError: doubler() got an unexpected keyword argument 'char'

In [16]:
#Furthermore, when executing z_triangle, it's doubler, hardcoded into
#the decorator, that's erroring out rather than z_triangle itself.
#This gives us the clue we need.
#Although it seems like we are passing an argument into the z_triangle,
#the decorator has modified z_triangle and overwritten it with doubler.
#Since doubler is written without arguments, it doesn't know what to do!

#Solution
def robust_doubler(func):
    def doubler(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return doubler

@robust_doubler
def z_square(char='z', n=3):
    for i in range(n):
        print(char*(n))
        
print('Normal execution')
z_square()
print('Modified execution')
z_square(char='zee')

Normal execution
zzz
zzz
zzz
zzz
zzz
zzz
Modified execution
zeezeezee
zeezeezee
zeezeezee
zeezeezee
zeezeezee
zeezeezee


In [17]:
#That's better.
#Still, the properties of z_square aren't what we might expect.
z_square.__name__

'doubler'

In [18]:
#Best practice would be to have our decorator transfer the properties
#of the original function along with modifying its behavior.
#Thankfully, there's a function in functools called wraps that that'll
#change the name, docstring, and module to that of the original function.
from functools import wraps
def robust_doubler_w_wraps(func):
    @wraps(func)
    def doubler(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return doubler

@robust_doubler_w_wraps
def o_square(char='o', n=3):
    for i in range(n):
        print(char*n)
        
o_square()
o_square(char='Oh ')
o_square.__name__

ooo
ooo
ooo
ooo
ooo
ooo
Oh Oh Oh 
Oh Oh Oh 
Oh Oh Oh 
Oh Oh Oh 
Oh Oh Oh 
Oh Oh Oh 


'o_square'

---
# Bonus: Decorator with arguments

In [19]:
#Decorators can also except arguments.
#Try to figure out what's going on by modifying the code.
def robust_repeater(n=3):
    def inner_decorator(func):
        @wraps(func)
        def repeated_func(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return repeated_func
    return inner_decorator
            

@robust_repeater(n=5)
def foo(msg):
    """This is a doc string."""
    print(msg)
    pass
    
foo('Yo')
foo.__doc__

Yo
Yo
Yo
Yo
Yo


'This is a doc string.'